<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 5:** Working with Large Documents</font>

<br>

In the previous notebook, we learned about running state chains and knowledge bases! By the end, we had all the tools necessary to do some simple dialog management and custom knowledge tracking. In this notebook, we will take the same ideas and move towards the space of large documents, considering what kinds of issues we will run into as we try to incorporate large files into our LLM contexts.

<br>

### **Learning Objectives:**

- Get familiar with document loaders and the kinds of utilities they might provide you.
- Learn how to parse large documents with limited context room by chunking the document and building up a knowledge base progressively.
- Understand how the progressive recontextualization, coersion, and consolidation of document chunks can be extremely useful, and also where it will encounter natural limitations.

<br>

### **Questions To Think About:**

- Looking at the chunks that come out of your ArxivParser, you'll notice that some of the chunks make little sense on their own or have been completely corrupted by the conversion to text. Is it doing a pass over the chunks to clean them up?
- Considering the document summarization workflow (or any similar workflow that processes through a large list of document chunks), how often should this happen, and when is it justifiable?

<br>

### **Environment Setup:**


In [1]:
## Necessary for Colab, not necessary for course environment
# %pip install -qq langchain langchain-nvidia-ai-endpoints gradio
# %pip install -qq arxiv pymupdf

import os
os.environ["NVIDIA_API_KEY"] = "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"

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:237: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(


[Model(id='01-ai/yi-large', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-yi-large'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='abacusai/dracarys-llama-3.1-70b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='ai21labs/jamba-1.5-large-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='ai21labs/jamba-1.5-mini-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='aisingapore/sea-lion-7b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-sea-lion-7b-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='baichuan-inc/baichu

In [4]:
## Useful utility method for printing intermediate states
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

----

<br>

## **Part 1:** Chatting with Documents

This notebook will begin a longer stream of discussion surrounding the use of LLMs to chat with documents. In a world where chat models are trained on giant repositories of public data and retraining them on custom data is prohibitively expensive, the idea of having an LLM reason about a set of PDFs or even a YouTube video opens up many opportunities!

- **Your LLM can have a modifiable knowledge base grounded in human-readible documents,** meaning that you can directly control what kinds of data it has access to and can instruct it to interact with it.

- **Your LLM can sort through and pull references directly from your document set.** With sufficient prompt engineering and instruction-following priors, you can force your models to only act based on the material you provide.

- **Your LLM can possibly even interact with your documents, making automatic modifications as necessary.** This opens up avenues in automatic content refinement and synthetic operations which will be explored later.

Listing out some possibilities is pretty easy, and from there you can let your imagination run wild... but we haven't gained the tools to do this quite yet, right?

<br>

#### **Naive Approach: Stuff Your Document**

Suppose you have some text documents (PDF, blog, etc.) and want to ask questions related to the contents of those documents. One approach you could try involves taking a representation of the document and feeding it all to a chat model! From a document perspective, this is known as [**document stuffing**](https://js.langchain.com/v0.1/docs/modules/chains/document/stuff/).

<!-- > <img src="https://drive.google.com/uc?export=view&id=14DRI_uDviqzqg14TKoIc8IlBc3Zsb8oO" width=800px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_stuff.png" width=800px/>
>
> From [**Stuff | LangChain**🦜️🔗](https://js.langchain.com/v0.1/docs/modules/chains/document/stuff/)

<br>

This may very well work if your model is strong enough and if your document is short enough, but it shouldn't be expected to work well for an entire document. Many modern LLMs have significant trouble working with long contexts due to training limitations. Nowadays large model deterioration isn't quite as catastrophic, but good instruction following is likely to fall apart pretty quickly regardless of which model you use (assuming you're accessing the raw model).

<br>

**The key issues you'll need to resolve with document reasoning are:**

- How do we split our documents into pieces that can be reasoned with?

- How can we find and consider these pieces efficiently as the size and number of documents increases?

This course will explore several approaches to address these issues while continuing to build up our LLM orchestration skills. ***This notebook will serve to expand out our previous running chain skills for more progressive reasoning formulations, whereas the next notebooks will introduce some new techniques to properly address retrieval at scale.*** Through this experience, we will continue to leverage cutting-edge open-source solutions to make our solutions standard and integratable.

Speaking of, the field of document loading frameworks has many strong options, and two major players will come up throughout the course:

- [**LangChain**](https://python.langchain.com/docs/get_started/introduction) provides a simple framework for connecting LLMs to your own data sources via general chunking strategies and strong incorporation with embedding frameworks/services. This framework has initially grown around its strong general support for LLM features, which signals its active strengths closer to the chain abstractions and agent coordination.

- [**LlamaIndex**](https://gpt-index.readthedocs.io/en/stable/) is a data framework for LLM applications to ingest, structure, and access private or domain-specific data. It has since branched out to include general LLM capabilities similar to LangChain, but as of now it is still strongest in addressing the document side of LLM components since its initial abstractions were centered around that problem.

It's recommended to read more about the unique strengths of both LlamaIndex and LangChain and pick the one that works best for you. Since LlamaIndex can be used *with* LangChain, the frameworks' unique capabilities can be leveraged together without too much issue. For the sake of simplicity, we will stick to LangChain in this course and will allow the [**NVIDIA/GenerativeAIExamples repository**](https://github.com/NVIDIA/GenerativeAIExamples/tree/main/RAG/notebooks) to explore deeper LlamaIndex options for those interested.


----

<br>

## **Part 2:** Loading Documents

LangChain provides a variety of [document loaders](https://python.langchain.com/docs/integrations/document_loaders) to facilitate the injestion of various document formats (HTML, PDF, code) from many different sources and locations (local storage, private s3 buckets, public websites, messaging APIs, etc.). These loaders query your data sources and return a `Document` object which contains the content and metadata, usually in a plain-text or otherwise human-readible format. There are plenty of document loaders already built and ready to use, with the first-party LangChain options listed [here](https://python.langchain.com/docs/integrations/document_loaders).

**In this example, we can load a research paper of our choice using one of the following LangChain loaders:**
- [`UnstructuredFileLoader`](https://python.langchain.com/docs/integrations/document_loaders/unstructured_file): Generally-useful file loader for arbitrary files; doesn't make too many assumptions about your document structure and is usually sufficient.
- [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv): A more specialized file-loader which can communicate with the Arxiv interface directly. [Just one example of many](https://python.langchain.com/docs/integrations/document_loaders), this will make some more assumptions about your data to yield nicer parsings and auto-fill metadata (useful when you have multiple documents/formats).

For our code example we will default to using `ArxivLoader` to load in one of either the [MRKL](https://arxiv.org/abs/2205.00445) or [ReAct](https://arxiv.org/abs/2210.03629) publication papers as you're likely to run into them at some point in your continued chat model research endeavors.

In [8]:
%%time
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import ArxivLoader

## Loading in the file

## Unstructured File Loader: Good for arbitrary "probably good enough" loader
# documents = UnstructuredFileLoader("llama2_paper.pdf").load()

## More specialized loader, won't work for everything, but simple API and usually better results
documents = ArxivLoader(query="2404.16130").load()  ## GraphRAG
documents = ArxivLoader(query="2404.03622").load()  ## Visualization-of-Thought
documents = ArxivLoader(query="2404.19756").load()  ## KAN: Kolmogorov-Arnold Networks
documents = ArxivLoader(query="2404.07143").load()  ## Infini-Attention
documents = ArxivLoader(query="2210.03629").load()  ## ReAct

CPU times: user 2.19 s, sys: 275 ms, total: 2.47 s
Wall time: 3.09 s


<br>

We can see from our import that we this connector gives us access to two different components:
- The `page_content` is the actual body of the document in some human-interpretable format.
- The `metadata` is relevant information about the document that is provided by the connector via its data source.

Below, we can check out the length of our document body to see what's inside, and will probably notice an intractable document length:

In [9]:
## Printing out a sample of the content
print("Number of Documents Retrieved:", len(documents))
print(f"Sample of Document 1 Content (Total Length: {len(documents[0].page_content)}):")
print(documents[0].page_content[:1000])

Number of Documents Retrieved: 1
Sample of Document 1 Content (Total Length: 110261):
Published as a conference paper at ICLR 2023
REACT: SYNERGIZING REASONING AND ACTING IN
LANGUAGE MODELS
Shunyu Yao∗
*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2
1Department of Computer Science, Princeton University
2Google Research, Brain team
1{shunyuy,karthikn}@princeton.edu
2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com
ABSTRACT
While large language models (LLMs) have demonstrated impressive performance
across tasks in language understanding and interactive decision making, their
abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action
plan generation) have primarily been studied as separate topics. In this paper, we
explore the use of LLMs to generate both reasoning traces and task-speciﬁc actions
in an interleaved manner, allowing for greater synergy between the two: reasoning
traces help the model induce, track, and update a

<br>

In contrast, the metadata will be much more conservatively-sized to the point of being viable context components for your favorite chat model:

In [10]:
pprint(documents[0].metadata)

{
    'Published': '2023-03-10',
    'Title': 'ReAct: Synergizing Reasoning and Acting in Language Models',
    'Authors': 'Shunyu Yao, Jeffrey Zhao, Dian Yu, Nan Du, Izhak Shafran, Karthik Narasimhan, Yuan Cao',
    'Summary': 'While large language models (LLMs) have demonstrated impressive capabilities\nacross tasks in 
language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought 
prompting) and acting (e.g.\naction plan generation) have primarily been studied as separate topics. In 
this\npaper, we explore the use of LLMs to generate both reasoning traces and\ntask-specific actions in an 
interleaved manner, allowing for greater synergy\nbetween the two: reasoning traces help the model induce, track, 
and update\naction plans as well as handle exceptions, while actions allow it to interface\nwith external sources, 
such as knowledge bases or environments, to gather\nadditional information. We apply our approach, named ReAct, to 
a diverse set of\nlanguage and decision making tasks and demonstrate its effectiveness over\nstate-of-the-art 
baselines, as well as improved human interpretability and\ntrustworthiness over methods without reasoning or acting
components.\nConcretely, on question answering (HotpotQA) and fact verification (Fever),\nReAct overcomes issues of
hallucination and error propagation prevalent in\nchain-of-thought reasoning by interacting with a simple Wikipedia
API, and\ngenerates human-like task-solving trajectories that are more interpretable than\nbaselines without 
reasoning traces. On two interactive decision making\nbenchmarks (ALFWorld and WebShop), ReAct outperforms 
imitation and\nreinforcement learning methods by an absolute success rate of 34% and 10%\nrespectively, while being
prompted with only one or two in-context examples.\nProject site with code: https://react-lm.github.io'
}

<br>

Though it may be tempting to accept the metadata format as-is and ignore the body entirely, there are a key selection of features that cannot be approached without diving into the full text:

- **The metadata is not guaranteed.** In the case of `arxiv`, paper abstracts, titles, authors, and date are necessary components of a submission, so being able to query them is not surprising. For an arbitrary PDF or webpage though, the same is not necessarily the case.
- **The agent will not be able to go deeper into the document content.** The summary is good to know and can be used as-is, but does not provide a straight-forward path to interacting with the body at any capacity (at least not from what we've learned).
- **The agent will still not be able to reason about too many documents at once.** Perhaps in the MRKL/ReAct example, you could combine those two summaries into one context and ask some questions. But what happens when you need to interact with 5 documents at once? What about an entire directory? Very soon, you will notice that your context window will be overloaded with information just to summarize or even list out the documents you're interested in!

----

<br>

## **Part 3:** Transforming The Documents

Once documents have been loaded, they often need to be transformed if we intend to pass them into our LLMs as context. One method of transformation is known as **chunking**, which breaks down large pieces of content into smaller segments. This technique is valuable because it helps [optimize the relevance of the content returned from the vector database](https://www.pinecone.io/learn/chunking-strategies/).

LangChain provides a [variety of document transformers](https://python.langchain.com/docs/integrations/document_transformers/) out of which we will use the [``RecursiveCharacterTextSplitter``](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter). This option will allow us to split our document based on a preference of natural stopping points.

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

## Some nice custom preprocessing
# documents[0].page_content = documents[0].page_content.replace(". .", "")
docs_split = text_splitter.split_documents(documents)

# def include_doc(doc):
#     ## Some chunks will be overburdened with useless numerical data, so we'll filter it out
#     string = doc.page_content
#     if len([l for l in string if l.isalpha()]) < (len(string)//2):
#         return False
#     return True

# docs_split = [doc for doc in docs_split if include_doc(doc)]
print(len(docs_split))

101


In [12]:
for i in (0, 1, 2, 15, -1):
    pprint(f"[Document {i}]")
    print(docs_split[i].page_content)
    pprint("="*64)

[Document 0]

Published as a conference paper at ICLR 2023
REACT: SYNERGIZING REASONING AND ACTING IN
LANGUAGE MODELS
Shunyu Yao∗
*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2
1Department of Computer Science, Princeton University
2Google Research, Brain team
1{shunyuy,karthikn}@princeton.edu
2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com
ABSTRACT
While large language models (LLMs) have demonstrated impressive performance
across tasks in language understanding and interactive decision making, their
abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action
plan generation) have primarily been studied as separate topics. In this paper, we
explore the use of LLMs to generate both reasoning traces and task-speciﬁc actions
in an interleaved manner, allowing for greater synergy between the two: reasoning
traces help the model induce, track, and update action plans as well as handle
exceptions, while actions allow it to interface with and

================================================================

[Document 1]

approach, named ReAct, to a diverse set of language and decision making tasks
and demonstrate its effectiveness over state-of-the-art baselines in addition to
improved human interpretability and trustworthiness. Concretely, on question
answering (HotpotQA) and fact veriﬁcation (Fever), ReAct overcomes prevalent
issues of hallucination and error propagation in chain-of-thought reasoning by
interacting with a simple Wikipedia API, and generating human-like task-solving
trajectories that are more interpretable than baselines without reasoning traces.
Furthermore, on two interactive decision making benchmarks (ALFWorld and
WebShop), ReAct outperforms imitation and reinforcement learning methods by
an absolute success rate of 34% and 10% respectively, while being prompted with
only one or two in-context examples.
1
INTRODUCTION
A unique feature of human intelligence is the ability to seamlessly combine task-oriented actions with
verbal reasoning (or inner speech, Alderson-Day & Fernyhough, 

...

We begin with knowledge-intensive reasoning tasks like multi-hop question answering and fact
veriﬁcation. As shown in Figure 1(1d), by interacting with a Wikipedia API, ReAct is able to
retrieve information to support reasoning, while also use reasoning to target what to retrieve next,
demonstrating a synergy of reasoning and acting.
3.1
SETUP
Domains
We consider two datasets challenging knowledge retrieval and reasoning: (1) Hot-
PotQA (Yang et al., 2018), a multi-hop question answering benchmark that requires reasoning
over two or more Wikipedia passages, and (2) FEVER (Thorne et al., 2018), a fact veriﬁcation
benchmark where each claim is annotated SUPPORTS, REFUTES, or NOT ENOUGH INFO, based
on if there exists a Wikipedia passage to verify the claim. In this work, we operate in a question-only
setup for both tasks, where models only receive the question/claim as input without access to support
paragraphs, and have to rely on their internal knowledge or retrieve knowledge via intera

================================================================

[Document -1]

located in the Bronx was published?
Thought:
Let’s think step by step.
The first year a scientific journal published by
an organization located in the Bronx was published was 1916.
Answer:
1916 (Label:
1909)
Failure: Label ambiguity
ReAct
Question:
What nationality is the man who developed the dictator game?
Thought 1:
I need to search dictator game, find the man who developed it, then find his
nationality.
Action 1:
Search[dictator game]
Observation 1:
The dictator game is a popular experimental instrument in social psychology
and economics...
Action 3:
Search[Amos Tversky]
Observation 3:
Amos Nathan Tversky...
Thought 4:
Amos Tversky is an Israeli cognitive and mathematical psychologist.
So the
man who developed the dictator game is Israeli.
Action 4:
Finish[Israeli] (label:
Israel-American)
CoT
Question:
Kurt Fuller from Stockton, CA appeared in which American Detective Comedy
Drama television series created by Steve Franks?
Thought:
Let’s think step by step.
Kurt Fuller from Stockt

================================================================

<br>

Our approach for chunking is pretty naive, but highlights the ease of getting at least something working for our application. We made some effort to keep the chunk size small so that our models are able to wield it effectively as context, but how are we going to reason about all of these pieces?

**When extending and optimizing this approach for an arbitrary set of documents, some potential options include:**

- Identifying logical breaks or synthesis techniques (manually, automatically, LLM-assisted, etc).
- Aiming to construct chunks that are rich in unique and relevant information, avoiding redundancy to maximize database utility.
- Customizing chunking to fit the document’s nature, ensuring the chunks are contextually relevant and cohesive.
- Including key concepts, keywords, or metadata snippets in each chunk for improved searchability and relevance in the database.
- Continuously assessing chunking effectiveness and be ready to adjust strategies for optimal balance between size and content richness.
- Considering a hierarchy system (implicitly-generated or explicitly-specified) to improve retrieval attempts.
    - If interested, please look over the [**LlamaIndex tree structures from the index guide**](https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide.html#tree-index) as a starting point.

----

<br>

## **Part 4: [Exercise]** Refining Summaries

To automatically reason about large documents, one potential idea might be to use LLMs to create a dense summary or knowledge base. Similar to how we maintained a running history of the conversation via slot-filling in the previous notebook, is there any problem with keeping a running history of an entire document?

In this section, we focus on an exciting application of LLMs: **automatically refining, coercing, and consolidating data en masse**. Specifically, we'll be implementing a simple but useful Runnable that uses a while loop and the running state chain formulation to summarize a set of document chunks. This process is commonly known as [**"document refinement"**](https://js.langchain.com/v0.1/docs/modules/chains/document/refine/) and is largely akin to our previous conversation-focused slot-filling exercise; the only difference is that now we're dealing with a large document instead of a growing chat history.

<!-- > <img src="https://drive.google.com/uc?export=view&id=1J2XR8Cc8YSkVJMiJCknMkgA02mBT8riZ" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_refine.png" width=1000px/>
>
> From [**Refine | LangChain**🦜️🔗](https://js.langchain.com/v0.1/docs/modules/chains/document/refine/)

<br>

#### **The DocumentSummaryBase Model**

Much like the `KnowledgeBase` class from the previous notebook, we can create a `DocumentSummaryBase` structure designed to encapsulate the essence of a document. The one below will use the `running_summary` field to query the model for a final summary while attempting to use the `main_ideas` and `loose_ends` fields as a bottleneck to keep the running summary from moving too fast. This is something we're going to have to enforce via prompt engineering, so the `summary_prompt` is also provided which shows how this information will be used. Feel free to modify it as necessary to make it work for your model of choice.

In [13]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from langchain_nvidia_ai_endpoints import ChatNVIDIA

from pydantic import BaseModel, Field
from typing import List
from IPython.display import clear_output


class DocumentSummaryBase(BaseModel):
    running_summary: str = Field("", description="Running description of the document. Do not override; only update!")
    main_ideas: List[str] = Field([], description="Most important information from the document (max 3)")
    loose_ends: List[str] = Field([], description="Open questions that would be good to incorporate into summary, but that are yet unknown (max 3)")


summary_prompt = ChatPromptTemplate.from_template(
    "You are generating a running summary of the document. Make it readable by a technical user."
    " After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything."
    " Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary."
    " The updated knowledge base keep all of the information from running_summary here: {info_base}."
    "\n\n{format_instructions}. Follow the format precisely, including quotations and commas"
    "\n\nWithout losing any of the info, update the knowledge base with the following: {input}"
)

<br>

We will also use this opportunity to bring back the `RExtract` function from the previous notebook:

In [14]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser


<br>

With this in mind, the following code invokes the running state chain in a for-loop to iterate over your documents! The only modification necessary should be the `parse_chain` implementation, which should pass the the state through a properly-configured `RExtract` chain from the last notebook. After this, the system should work decently to maintain a running summary of the document (though some tweaking of the prompt may be required depending on the model used).

In [15]:
latest_summary = ""

## TODO: Use the techniques from the previous notebook to complete the exercise
def RSummarizer(knowledge, llm, prompt, verbose=False):
    '''
    Exercise: Create a chain that summarizes
    '''
    ###########################################################################################
    ## START TODO:

    def summarize_docs(docs):        
        ## TODO: Initialize the parse_chain appropriately; should include an RExtract instance.
        ## HINT: You can get a class using the <object>.__class__ attribute...
        parse_chain = RunnableAssign({'info_base' : (lambda x: None)})
        ## TODO: Initialize a valid starting state. Should be similar to notebook 4
        state = {}

        global latest_summary  ## If your loop crashes, you can check out the latest_summary
        
        for i, doc in enumerate(docs):
            ## TODO: Update the state as appropriate using your parse_chain component

            assert 'info_base' in state 
            if verbose:
                print(f"Considered {i+1} documents")
                pprint(state['info_base'])
                latest_summary = state['info_base']
                clear_output(wait=True)

        return state['info_base']
        
    ## END TODO
    ###########################################################################################
    
    return RunnableLambda(summarize_docs)

# instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1").bind(max_tokens=4096)
instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1").bind(max_tokens=4096)
instruct_llm = instruct_model | StrOutputParser()

## Take the first 10 document chunks and accumulate a DocumentSummaryBase
summarizer = RSummarizer(DocumentSummaryBase(), instruct_llm, summary_prompt, verbose=True)
summary = summarizer.invoke(docs_split[:15])

AssertionError: 

In [ ]:
pprint(latest_summary)

----

<br>

## **Part 5:** Synthetic Data Processing

As we conclude our exploration of document summarization using LLMs, it's important to acknowledge the broader context and potential challenges. While we've demonstrated a viable method for extracting concise, meaningful summaries, let's consider why such an approach is crucial and the complexities it entails.

#### **Generality of Refinement**

It's important to note that this "progressive summarization" technique is merely a starter chain that makes few assumptions about the initial data and desired output format. The same technique can be expanded far and wide to generate synthetic refinements with known metadata, active assumptions, and downstream objectives in mind.

**Consider these potential applications:**

1. **Aggregating Data**: Constructing structures that transform raw data from document chunks into coherent, useful summaries.
2. **Categorization and Sub-topic Analysis**: Creating systems that categorize insights from chunks into defined categories, tracking emerging sub-topics within each.
3. **Consolidation into Dense Informational Chunks**: Refining these structures to distill insights into compact segments, enriched with direct quotes for deeper analysis.

These applications hint at the creation of a **domain-specific knowledge graph** which can be accessed and traversed by a conversational chat model. Some utilities already exist to generate these automatically via tools like [**LangChain Knowledge Graphs**](https://python.langchain.com/docs/tutorials/graph/). Though you might need to develop hierarchical structures and tools to both construct and traverse such a structure, it is a viable option when you can properly refine a sufficient knowledge graph for your use case! For this intereted in more advanced knowledge graph construction techniques which rely on larger systems and vector similarity, we found the [**LangChain x Neo4j Article**](https://blog.langchain.dev/using-a-knowledge-graph-to-implement-a-devops-rag-application/) to be of interest.

### **Addressing the Challenges of Large-Scale Data Processing**

While our approach opens up exciting possibilities, it's not without its challenges, especially when dealing with large volumes of data:

- **Generic Preprocessing Limitations**: While summarization is relatively straightforward, developing hierarchies that are universally effective across various contexts is challenging.

- **Granularity and Navigation Costs**: Achieving detailed granularity in a hierarchy can be resource-intensive, requiring sophisticated consolidation or extensive branching to maintain manageable context sizes per interaction.

- **Dependency on Precise Instruction Execution**: Navigating such a hierarchy with our current tools would rely heavily on powerful instruction-tuned models with strong prompt engineering. The inference latency and the risk of errors in argument prediction can be significant, so using LLMs for this could be a challenge.

As you progress through the course, keep track of how these challenges get addressed with subsequent techniques. 

-----

<br>

## **Part 6:** Wrap-Up

The goal of this notebook was to introduce the problems and techniques surrounding large document handling for chat models. In the next notebook, we will investigate a complementary tool with a very different set of pros and cons; **semantic retrieval with embedding models.**

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.
2. **[Optional]** This notebooks includes some fundamental document processing chains, but does not touch upon [Map Reduce](https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/) chains which are also very useful and build on roughly the same intuitions. These are a good next step, so please check them out!

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>